In [1]:
import os
import numpy as np

%load_ext autoreload
%autoreload 2

from datetime import datetime, timezone
from assignsvp import assignsvp as asvp
from cli import cli

In [2]:
class SoundVelProfile:
    """Class handling SVP data"""
    
    def __init__(self):
        self.identific = ''
        self.sn = ''
        self.fname = ''
        self.datetime = ''
        self.station = ''
        self.lat = ''
        self.lon = ''
        
        self.press = []
        self.temp = []
        self.sv = []

In [3]:
svp_path = r'D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW'
svp_meta = r'D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\ANS54_SVP_Metadata.csv'


svp_list = cli.search(ext='.TXT', path=svp_path)

Searching *.TXT files in directory:D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW
Scanned filesets are: 

0 D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW\V000053.TXT
1 D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW\V000056.TXT
2 D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW\V000059.TXT
3 D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW\V000061.TXT


In [4]:
svp_objs = []

with open(svp_meta, 'r') as file1:
    file_content = file1.read().splitlines()
    
    for line in file_content[1:]:
        line_content = line.split(',')
        svp_obj = SoundVelProfile()
        
        svp_obj.identific = line_content[0]
        svp_obj.sn = line_content[1]
        svp_obj.fname = line_content[2]
        svp_obj.datetime = line_content[3]
        svp_obj.station = line_content[4]
        svp_obj.lat = line_content[5]
        svp_obj.lon = line_content[6]
        
        svp_objs.append(svp_obj)

In [21]:
for svp_file in svp_list:
    filename = os.path.splitext(os.path.basename(svp_file))[0]
    print(filename)
    print(svp_file)
    
    sn = ''
    
    pressure = []
    temperature = []
    sound_speed = []
    
    with open(svp_file) as file2:
        file_content = file2.read().splitlines()
        
        for line in file_content[0:9]:
            if line.startswith('MiniSVP'):
                sn = line.split(' ')[-1]
                
        for line in file_content[9:]:
            line_content = line.split()
            
            if line_content[0]:
                pressure.append(line_content[0])
                temperature.append(line_content[1])
                sound_speed.append(line_content[2])
            
    for svp_obj in svp_objs:
        if filename == svp_obj.fname and sn == svp_obj.sn:
            svp_obj.press = pressure
            svp_obj.temp = temperature
            svp_obj.sv = sound_speed
    
        
        
        

V000053
D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW\V000053.TXT
V000056
D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW\V000056.TXT
V000059
D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW\V000059.TXT
V000061
D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск\SVP_Data\RAW\V000061.TXT


In [22]:
def save_pos_file(path,cruise):
    filename = os.path.join(path, 'E090005O2_' + cruise + '_H14_POS.csv')
    
    with open(filename, 'w') as file3:
        file3.write('rec_num;identific;station;LONG;LAT;DATE\n')
        
        for num, svp_obj in enumerate(svp_objs):
            file3.write(f'{num};{svp_obj.identific};{svp_obj.station:12};{float(svp_obj.lon):.8f};{float(svp_obj.lat):.8f};{svp_obj.datetime}\n')
        
def save_svp_file(path, cruise, svp_obj:SoundVelProfile):
    filename = os.path.join(path, f'E090005O2_{cruise}_H14_{int(svp_obj.identific):03d}.csv')

    with open(filename, 'w') as file4:
        file4.write('rec_num;identific;station;SN;Pres;Temp;SoundVel\n')
        
        for num, _ in enumerate(svp_obj.press):
            file4.write(f'{num};{svp_obj.identific};{svp_obj.station:12};{svp_obj.sn};{svp_obj.press[num]};{svp_obj.temp[num]};{svp_obj.sv[num]}\n')

In [23]:
path_data = r'D:\aa_yandexcloud\aa_cruises\ANS54\Отчёт в Обнинск'

save_pos_file(path=path_data, cruise='ANS_54')


In [24]:
for svp_obj in svp_objs:
    save_svp_file(path_data, 'ANS_54', svp_obj)